# 1 Read data & basic text prerpocessing

In [1]:
import re
import sys
import shutil as sh
import os

In [2]:
#path = "/home/yingjie/Desktop/Machine_Learning/Assignment4/data/"
#path2 = "/home/yingjie/Desktop/Machine_Learning/Assignment4/data_clean/"
path = "/Users/Pan/Google Drive/Data Science/Machine Learning/ML_Assignment4/data/"
path2 = "/Users/Pan/Google Drive/Data Science/Machine Learning/ML_Assignment4/data_clean/"

In [3]:
def data_preprocess(path,thefile):
    # Replace all numbers with "NUM" special character
    num_dec = re.compile("[\d]+\.[\d]+")
    num_comma = re.compile("[\d]+,[\d]+")
    num_reg = re.compile("[\d]+")

    # Detect non-English characters
    non_eng = re.compile("[^a-zA-Z\n ]")
    
    #open file
    file=open(os.path.join(path,thefile), "r", encoding="iso-8859-15")
    text = file.read().lower()
    file.close()
    
    # Replace various forms of numbers with NUM character
    text = num_dec.sub("NUM", text)
    text = num_comma.sub("NUM", text)
    text = num_reg.sub("NUM", text)

    # Replace all non-English characters with a space
    text = non_eng.sub(" ", text)

    # Replace double spaces with a single space
    text = text.replace("  ", " ")
    
    # Split the file into reviews based on \n
    text = text.split("\n")
        
    return(text)

In [4]:
train=data_preprocess(path,"down_sampled_reviews/train_tiny.txt")
valid=data_preprocess(path,"down_sampled_reviews/valid_tiny.txt")

## DONT RUN THIS SESSION ##
# 2 Create vocabulary - demonstration only


reference source:https://stackoverflow.com/questions/40661684/tensorflow-vocabularyprocessor

In [5]:
import numpy as np
from tensorflow.contrib import learn

/Users/Pan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
num_of_reviews=2
small_train=''
for i in range(num_of_reviews):
    small_train+=train[i]+" END "
small_train=[small_train]
small_train

['i don t know why but flexaril just isn t doing the trick for me anymore i m shocked by how well this product works for muscle spasms i suppose the valerian and passiflora which are also mental relaxants along with the magnesium for the cramping help give you that over all feeling of ahhh  i know not everything works for everyone and it s a relatively expensive purchase if it doesn t if you re need of one of their other vitamins order those and they ll send along a free trial of the valerin and you can find out for yourself that s how i got to experience it i ordered their cinnamon for blood sugar support and got the sample i also take the valerin at night with schiff melatonin plus total relaxation comes swiftly and long vivid dreams follow  a side effect of melatonin i really enjoy i like this product so much i ve been passing it out like a pusher telling people try this it s awesome  END not the most innovative concept in marketing but galdwell did a very good job in explaining a c

In [7]:
max_document_length=len(small_train[0].split(" "))
max_document_length

213

In [8]:
## Create the vocabularyprocessor object, setting the max lengh of the documents.
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)

In [ ]:
## Transform the documents using the vocabulary.
train_transformed = np.array(list(vocab_processor.fit_transform(small_train)))
train_transformed

In [ ]:
train_transformed=[np.trim_zeros(train_transformed[0], 'b')]
train_transformed

In [ ]:
## Extract word:id mapping from the object.
vocab_dict = vocab_processor.vocabulary_._mapping
vocab_dict['END'] = -1
vocab_dict.items()

In [ ]:
# -1 is in 
train_transformed2=np.array(list(vocab_processor.fit_transform(small_train)))
train_transformed2=[np.trim_zeros(train_transformed2[0], 'b')]
train_transformed2

In [ ]:
## Sort the vocabulary dictionary on the basis of values(id).
## Both statements perform same task.
#sorted_vocab = sorted(vocab_dict.items(), key=operator.itemgetter(1))
sorted_vocab = sorted(vocab_dict.items(), key = lambda x : x[1])
## Treat the id's as index into list and create a list of words in the ascending order of id's
## word with id i goes at index i of the list.
vocabulary = list(list(zip(*sorted_vocab))[1])
print(vocabulary)

In [ ]:
seq_size = 2
trainX, trainY = [], []
max_document_length=len(train_transformed2[0])
for i in range(max_document_length): 
    #[0,1,2,3,4,5,-1,1,2,3,4,5]
    if -1 not in train_transformed2[0][i:i+seq_size+1]:
        trainX.append(np.expand_dims(train_transformed2[0][i:i+seq_size], axis=1).tolist())
        trainY.append(train_transformed2[0][i+1:i+seq_size+1])

# Inspect trainX and trainY
print("X:\n",'\n'.join([str(item) for item in trainX[0:5]]))
print("Y:\n",'\n'.join([str(item) for item in trainY[0:5]]))

In [ ]:
size_vocab = len(vocabulary)

# 2b Create vocabulary through a function

In [9]:
import numpy as np
from tensorflow.contrib import learn

In [10]:
def textlst_2_vocab(textlst,num_of_reviews=None,seq_size = 2):
    if num_of_reviews is None:
        num_of_reviews=len(textlst)
        
    small_train=''
    for i in range(num_of_reviews):
        small_train+=textlst[i]+" END "
    small_train=[small_train]
    
    max_document_length=len(small_train[0].split(" "))
    
    ## Create the vocabularyprocessor object, setting the max lengh of the documents.
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    
    ## Transform the documents using the vocabulary.
    train_transformed = np.array(list(vocab_processor.fit_transform(small_train)))
    
    ## Extract word:id mapping from the object.
    vocab_dict = vocab_processor.vocabulary_._mapping
    vocab_dict['END'] = -1
    
    train_transformed2=np.array(list(vocab_processor.fit_transform(small_train)))
    train_transformed2=[np.trim_zeros(train_transformed2[0], 'b')]
    
    #sorted_vocab = sorted(vocab_dict.items(), key=operator.itemgetter(1))
    sorted_vocab = sorted(vocab_dict.items(), key = lambda x : x[1])
    ## Treat the id's as index into list and create a list of words in the ascending order of id's
    ## word with id i goes at index i of the list.
    vocabulary = list(list(zip(*sorted_vocab))[1])
    
    trainX, trainY = [], []
    max_document_length=len(train_transformed2[0])
    for i in range(max_document_length): 
        #[0,1,2,3,4,5,-1,1,2,3,4,5]
        if -1 not in train_transformed2[0][i:i+seq_size+1]:
            trainX.append(np.expand_dims(train_transformed2[0][i:i+seq_size], axis=1).tolist())
            trainY.append(train_transformed2[0][i+1:i+seq_size+1])
    
    size_vocab = len(vocabulary)
    
    return trainX,trainY,size_vocab

In [11]:
num_of_reviews=None
seq_size=2
trainX,trainY,size_vocab=textlst_2_vocab(train,num_of_reviews=num_of_reviews,seq_size = seq_size)
print(size_vocab)
print("X:\n",'\n'.join([str(item) for item in trainX[0:5]]))
print("Y:\n",'\n'.join([str(item) for item in trainY[0:5]]))

32099
X:
 [[1], [2]]
[[2], [3]]
[[3], [4]]
[[4], [5]]
[[5], [6]]
Y:
 [2 3]
[3 4]
[4 5]
[5 6]
[6 7]


In [12]:
num_of_reviews_valid = None
validX,validY,size_vocab_val=textlst_2_vocab(valid,num_of_reviews=num_of_reviews_valid,seq_size = seq_size)
print(size_vocab_val)
print("X:\n",'\n'.join([str(item) for item in validX[0:5]]))
print("Y:\n",'\n'.join([str(item) for item in validY[0:5]]))

24303
X:
 [[1], [2]]
[[2], [3]]
[[3], [4]]
[[4], [5]]
[[5], [6]]
Y:
 [2 3]
[3 4]
[4 5]
[5 6]
[6 7]


# TF setup

    # output dense layer:
    # convert W from [# hidden, 1] to [# training examples, # hidden, 1]
    # step 1: add a new dimension at index 0 using tf.expand_dims
    #w_exp= tf.expand_dims(W_out,0)
    # step 2: duplicate W for 'num_examples' times using tf.tile
    #W_repeated = tf.tile(w_exp,[num_examples,1,1])
    
    # Dense Layer calculation: 
    # [# training examples, sequence length, # hidden] *
    # [# training examples, # hidden, 1] = [# training examples, sequence length]
    
    # Actually, y_pred: [# training examples, sequence length, 1]
    # Remove last dimension using tf.squeeze
    #y_pred = tf.add(tf.matmul(logit, W_repeated),b_out)
    #y_pred = tf.squeeze(y_pred)    # [# training examples, sequence length]
    #y_pred = tf.reshape(y_pred)
    
    #num_examples= tf.cast(num_examples, tf.float32)

In [13]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.preprocessing import MinMaxScaler
import timeit

In [14]:
# Build computational graph
input_dim=1 # dim > 1 for multivariate time series
hidden_dim=100 # number of hiddent units h
graph = tf.Graph()

with graph.as_default():
    # input place holders
    # input Shape: [# training examples, sequence length, # features]
    x = tf.placeholder(tf.float32,[None,seq_size,input_dim])
    print("input shape:", x.shape)
    
    # label Shape: [# training examples, sequence length]
    y = tf.placeholder(tf.int32,[None,seq_size])
    print("ground truth shape:", y.shape)
    
    num_examples = tf.shape(x)[0]
    
    # RNN Network
    cell = rnn.BasicRNNCell(hidden_dim)
    
    # RNN output Shape: [# training examples, sequence length, # hidden] 
    outputs, _ = tf.nn.dynamic_rnn(cell,x,dtype=tf.float32)
    print("output from tf,nn.dynamic_rnn shape", outputs.shape)
    
    #outputs = tf.reshape(outputs, [num_examples*seq_size, hidden_dim])
    print("reshaped output from RNN shape:", outputs.shape)
    
    # weights for output dense layer (i.e., after RNN)
    # W shape: [# hidden, 1]
    W_out = tf.Variable(tf.random_normal([hidden_dim,size_vocab]),name="w_out")
    print("weights shape:", W_out.shape)
    # b shape: [1]
    b_out = tf.Variable(tf.random_normal([size_vocab]),name="b_out")
    print("bias shape",b_out.shape)
    
    # logit shape [# training examples, vocab_size] 
    outputs_reshaped = tf.reshape(outputs, [num_examples*seq_size, hidden_dim])
    print("fully connected and reshaped outputs:", outputs_reshaped) 
    
    network_output = ((tf.matmul(outputs_reshaped,W_out) + b_out))
    #print(network_output)
          
    y_pred = tf.reshape(network_output,[num_examples, seq_size, size_vocab])
    print("final logits shape:", y_pred.shape)
    
    # Cost & Training Step
    #cost = -tf.reduce_sum(y*tf.log(y_pred)) #cross_entropy
    weights = tf.ones([num_examples,seq_size])
    loss = tf.contrib.seq2seq.sequence_loss(y_pred,y,weights) 
    #cost = tf.reduce_sum(loss)/num_examples
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)


input shape: (?, 2, 1)
ground truth shape: (?, 2)
output from tf,nn.dynamic_rnn shape (?, 2, 100)
reshaped output from RNN shape: (?, 2, 100)
weights shape: (100, 32099)
bias shape (32099,)
fully connected and reshaped outputs: Tensor("Reshape:0", shape=(?, 100), dtype=float32)
final logits shape: (?, 2, 32099)


In [16]:
max_itr=2
batch_size=50
batch_num=len(trainX)//batch_size
print("batch_num: " +str(batch_num))

batch_num: 12406


In [19]:
# Run Session

with tf.Session(graph=graph) as sess:
    # initialize variables
    sess.run(tf.global_variables_initializer())
    # Run for 1000 iterations (1000 is arbitrary, need a validation set to tune!)
    print('Training...')
    start=timeit.default_timer()
    print("start time: %10.5f"%(start))

    for i in range(max_itr): 
        print("iter %3d" %(i))  
        print ("max batch num: "+str(batch_num))
        for NUM in range(1,batch_num):
            batch_trainX=trainX[(NUM-1)*batch_size:NUM*batch_size]
            batch_trainY=trainY[(NUM-1)*batch_size:NUM*batch_size]
            
            _, train_err = sess.run([train_op,loss],feed_dict={x:batch_trainX,y:batch_trainY})

            if  NUM % 100 == 0: 
                print("  ##batch NUM %6d :" %(NUM))
                print('    step, train err= %6d: %8.5f' % (i+1,train_err))
            
            #print('  step, test err= %6d: %8.5f' % (i+1,test_err))
            
    print("  ##batch NUM %6d :" %(NUM))
    print('    step, train err= %6d: %8.5f' % (i+1,train_err))
    
    ##### Test trained model on training data
    predicted_vals_all= sess.run(y_pred,feed_dict={x:trainX})
    
    # Get predicted value in each predicted sequence:
    predicted_vals = np.argmax(predicted_vals_all,2) 

    #print("Dimension check for train:"+str(np.size(predicted_vals_all,0)==len(trainX)))
    myaccuracy=sum([el[1] for el in predicted_vals==trainY])/np.size(predicted_vals_all,0)
    print('my training accuracy = %8.5f' % myaccuracy)
    
    end=timeit.default_timer()
    print("Training time : %10.5f"%(end-start))
    ####### Test trained model on testing data
    print('---------------------------------')
    test_err = sess.run(loss,feed_dict={x:validX,y:validY})
    print('test err=  %8.5f' % (train_err))
    
    # testing error:
    predicted_vals_all_test= sess.run(y_pred,feed_dict={x:validX})
    
    # Get predicted value in each predicted sequence:
    predicted_vals_test = np.argmax(predicted_vals_all_test,2) 
    
    # testing accuracy:
    myaccuracy_test=sum([el[1] for el in predicted_vals_test==validY])/np.size(predicted_vals_all_test,0)
    print('my testing accuracy = %8.5f' % myaccuracy_test)
    print('---------------------------------')
    

Training...
start time: 161524.74648
iter   0
max batch num: 12406
  ##batch NUM    100 :
    step, train err=      1: 12.96130
  ##batch NUM    200 :
    step, train err=      1:  9.50537


KeyboardInterrupt: 

## Compare performance of RNNs, LSTMs, GRUs

In [ ]:
import timeit
# Encapsulate the entire prediction problem as a function
def build_and_predict(trainX,trainY,testX,cell,cellType,input_dim=1,hidden_dim=100,seq_size = 12,max_itr=200):

    graph = tf.Graph()
    with graph.as_default():
        # input place holders
        # input Shape: [# training examples, sequence length, # features]
        x = tf.placeholder(tf.float32,[None,seq_size,input_dim])
        # label Shape: [# training examples, sequence length]
        y = tf.placeholder(tf.float32,[None,seq_size])
        
        
        
        # RNN output Shape: [# training examples, sequence length, # hidden] 
        outputs, _ = tf.nn.dynamic_rnn(cell,x,dtype=tf.float32)
        
        
        # weights for output dense layer (i.e., after RNN)
        # W shape: [# hidden, 1]
        W_out = tf.Variable(tf.random_normal([hidden_dim,1]),name="w_out") 
        # b shape: [1]
        b_out = tf.Variable(tf.random_normal([1]),name="b_out")
    
        # output dense layer:
        num_examples = tf.shape(x)[0] 
        # convert W from [# hidden, 1] to [# training examples, # hidden, 1]
        # step 1: add a new dimension at index 0 using tf.expand_dims
        w_exp= tf.expand_dims(W_out,0)
        # step 2: duplicate W for 'num_examples' times using tf.tile
        W_repeated = tf.tile(w_exp,[num_examples,1,1])
        
        # Dense Layer calculation: 
        # [# training examples, sequence length, # hidden] *
        # [# training examples, # hidden, 1] = [# training examples, sequence length]
        
        y_pred = tf.matmul(outputs,W_repeated)+b_out
        # Actually, y_pred: [# training examples, sequence length, 1]
        # Remove last dimension using tf.squeeze
        y_pred = tf.squeeze(y_pred)
        
        # Cost & Training Step
        cost = tf.reduce_mean(tf.square(y_pred-y))
        train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
        
        # Run Session
    with tf.Session(graph=graph) as sess:
        # initialize variables
        sess.run(tf.global_variables_initializer())
        # Run for 1000 iterations (1000 is arbitrary, need a validation set to tune!)
        start=timeit.default_timer()
        print('Training %s ...'%cellType)
        for i in range(max_itr): # If we train more, would we overfit? Try 10000
            _, train_err = sess.run([train_op,cost],feed_dict={x:trainX,y:trainY})
            if i==0:
                print('  step, train err= %6d: %8.5f' % (0,train_err)) 
            elif  (i+1) % 100 == 0: 
                print('  step, train err= %6d: %8.5f' % (i+1,train_err)) 
        end=timeit.default_timer()        
        print("Training time : %10.5f"%(end-start))
         # Test trained model on training data
        predicted_vals_all= sess.run(y_pred,feed_dict={x:testX}) 
        # Get last item in each predicted sequence:
        predicted_vals = predicted_vals_all[:,seq_size-1]
      
    
    return predicted_vals